# Ingenieria Machine Learning

## Librerias

In [1]:
import pandas as pd
from pandas import json_normalize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel
import gensim
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


## Rutas y DataFrames

In [2]:
#Sitios: Informacion sobre los restaurantes mexicanos en el estado de florida (Google)
#Scraping_sitios_google: Informacion extraida mediante web scraping (Yelp)
#user:  Cantidad de review por cada usuario y el promedio de estrellas (Yelp)
#Reviews_florida: Diferentes reviews de usuarios sobre los restaurantes mexicanos de florida (Google)
#reviews: Diferenetes tipos de reviews sobre restaurantes de florida (Yelp)
#business: Caracteristicas de todos los restaurantes de comida mexicana de Florida (Yelp)

In [3]:
ruta1= r"C:\Users\juanc.DESKTOP-LGMDQP1\OneDrive\Documentos\Proyecto Henry Final\ProyectoG4-Google_Yelp\Data\Parquet\Sitios.PARQUET"
ruta2=r"C:\Users\juanc.DESKTOP-LGMDQP1\OneDrive\Documentos\Proyecto Henry Final\ProyectoG4-Google_Yelp\Data\Parquet\Scraping_sitios_google.PARQUET"
ruta3=r"C:\Users\juanc.DESKTOP-LGMDQP1\OneDrive\Documentos\Proyecto Henry Final\ProyectoG4-Google_Yelp\Data\Parquet\user.PARQUET"
ruta4=r"C:\Users\juanc.DESKTOP-LGMDQP1\OneDrive\Documentos\Proyecto Henry Final\ProyectoG4-Google_Yelp\Data\Parquet\Reviews_florida.PARQUET"
ruta5=r"C:\Users\juanc.DESKTOP-LGMDQP1\OneDrive\Documentos\Proyecto Henry Final\ProyectoG4-Google_Yelp\Data\Parquet\review.PARQUET"
ruta6=r"C:\Users\juanc.DESKTOP-LGMDQP1\OneDrive\Documentos\Proyecto Henry Final\ProyectoG4-Google_Yelp\Data\Parquet\business.PARQUET"


In [4]:
dfSitios=pd.read_parquet(ruta1)
dfScraping=pd.read_parquet(ruta2)
dfuser=pd.read_parquet(ruta3)
dfReviewsFL=pd.read_parquet(ruta4)
dfreview=pd.read_parquet(ruta5)
dfbusiness=pd.read_parquet(ruta6)


## Se concatena los datos de Yelp y los de Google para dejar una sola estructura de datos

In [5]:
modelo_yelpdf = pd.merge(dfbusiness[['business_id', 'name', 'city', 'latitude', 'longitude', 'stars business']],
                               dfreview[['business_id', 'text', 'stars review', 'fecha review']],
                               on='business_id')

In [6]:
modelo_googledf = pd.merge(dfReviewsFL[['business_name', 'rating', 'city', 'gmap_id', 'date', 'text']],
                                    dfSitios[['gmap_id','latitude', 'longitude', 'avg_rating']],
                                    on='gmap_id')


In [7]:
modelo_googledf = modelo_googledf.rename(columns={
    'business_id': 'business_id',
    'business_name': 'business_name',
    'city': 'city',
    'latitude': 'latitude',
    'longitude': 'longitude',
    'avg_rating': 'business_rating',
    'text': 'review_text',
    'rating': 'review_stars',
    'date': 'review_date'
})

In [8]:
modelo_yelpdf.rename(columns={'business_id': 'business_id', 'name': 'business_name', 'city': 'city', 'latitude': 'latitude', 'longitude': 'longitude', 'stars business': 'business_rating', 'text': 'review_text', 'stars review': 'review_stars', 'fecha review': 'review_date'}, inplace=True)


In [9]:
#hacer un append de las tablas modelo_regresion_yelpdf y modelo_regresion_googledf y guardar el resultado en un data frame de pandas llamado modelo_regresiondf y luego pasar a un archivo parquet llamado modelo_regresion.csv
dfFinal = pd.concat([modelo_yelpdf, modelo_googledf])

## Analisis de sentimientos

In [10]:
# Inicializar el analizador de sentimientos de NLTK
sid= SentimentIntensityAnalyzer()

In [11]:
#Analisis de sentimiento para el archivo review

# Retrona 0 si es una reseña negativa
# Retrona 1 si es una reseña nula o neutra
# Retrona 2 si es una reseña positiva

def analyze_sentiment(text):
    if text is None or pd.isna(text):
        return 1 
    else:
        
        sentiment_score = sid.polarity_scores(text)['compound']
        if sentiment_score < -0.05:
            return 0  
        elif sentiment_score > 0.05:
            return 2  
        else:
            return 1  

dfFinal['sentiment_analysis'] = dfFinal['review_text'].apply(analyze_sentiment)

In [12]:
dfFinal

,business_id,business_name,city,latitude,longitude,business_rating,review_text,review_stars,review_date,gmap_id,sentiment_analysis
0,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,27.894167,-82.779866,3.0,The New Chipotle Mexican Grill conveniently lo...,4,2017-08-30,NaN,2
1,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,27.894167,-82.779866,3.0,I am a chipotle enthusiast but from the first ...,5,2018-01-18,NaN,2
2,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,27.894167,-82.779866,3.0,Went into this location about an hour before t...,5,2018-09-08,NaN,2
3,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,27.894167,-82.779866,3.0,I come to this chipotle often and it's one of ...,5,2019-06-24,NaN,2
4,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,27.894167,-82.779866,3.0,"The food is good, as per usual at any Chipotle...",2,2018-06-20,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...
15639,NaN,Taco Bus Miami,Miami,25.760594,-80.409314,4.1,None,1,2020-04-01,0x88d9bf290ee132e7:0x771f3f6950e264db,1
15640,NaN,Taco Bus Miami,Miami,25.760594,-80.409314,4.1,None,5,2021-03-31,0x88d9bf290ee132e7:0x771f3f6950e264db,1
15641,NaN,Taco Bus Miami,Miami,25.760594,-80.409314,4.1,None,5,2020-09-14,0x88d9bf290ee132e7:0x771f3f6950e264db,1
15642,NaN,Taco Bus Miami,Miami,25.760594,-80.409314,4.1,None,5,2020-09-28,0x88d9bf290ee132e7:0x771f3f6950e264db,1


## Se almacena en un dataframe solo los datos de la cadena Taco Bell

In [13]:
filtro= dfFinal["business_name"] == "Taco Bell"
dfTacoBell=dfFinal[filtro]

In [14]:
dfTacoBell

,business_id,business_name,city,latitude,longitude,business_rating,review_text,review_stars,review_date,gmap_id,sentiment_analysis
232,H2nFgEFitRk7ltE8IRH0xA,Taco Bell,Tampa,27.944918,-82.523213,2.5,This Taco Bell lives up to the standards that ...,5,2017-05-26,NaN,2
233,H2nFgEFitRk7ltE8IRH0xA,Taco Bell,Tampa,27.944918,-82.523213,2.5,"It's Taco Bell. It's crap food, terrible servi...",1,2018-01-01,NaN,0
234,H2nFgEFitRk7ltE8IRH0xA,Taco Bell,Tampa,27.944918,-82.523213,2.5,"Was staying at hotel across street, got in lat...",1,2018-05-13,NaN,0
235,H2nFgEFitRk7ltE8IRH0xA,Taco Bell,Tampa,27.944918,-82.523213,2.5,Now I see why this location has such terrible ...,1,2018-07-03,NaN,0
236,H2nFgEFitRk7ltE8IRH0xA,Taco Bell,Tampa,27.944918,-82.523213,2.5,"Waited 45 minutes at the drive thru, surely em...",1,2017-12-27,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
14328,NaN,Taco Bell,Orange Park,30.185016,-81.735113,3.0,None,2,2021-05-19,0x88e5c67d65294965:0x86e12ae83b3ab99a,1
14329,NaN,Taco Bell,Orange Park,30.185016,-81.735113,3.0,None,3,2021-03-09,0x88e5c67d65294965:0x86e12ae83b3ab99a,1
14330,NaN,Taco Bell,Orange Park,30.185016,-81.735113,3.0,None,1,2021-04-09,0x88e5c67d65294965:0x86e12ae83b3ab99a,1
14331,NaN,Taco Bell,Orange Park,30.185016,-81.735113,3.0,None,5,2019-06-28,0x88e5c67d65294965:0x86e12ae83b3ab99a,1


## Se extraen algunos datos relevantes de Taco Bell y de la competencia 

### Analisis de la competencia

In [15]:
porcentajes = (dfFinal["sentiment_analysis"].value_counts(normalize=True) * 100) 
porcentajes = porcentajes.round(2).astype(str) + '%'
porcentajes

sentiment_analysis
2    75.87%
0    12.34%
1    11.78%
Name: proportion, dtype: object

### Analisis Taco Bell

In [16]:
porcentajes2 = (dfTacoBell["sentiment_analysis"].value_counts(normalize=True) * 100) 
porcentajes2 = porcentajes2.round(2).astype(str) + '%'
porcentajes2

sentiment_analysis
1    36.62%
2    32.28%
0     31.1%
Name: proportion, dtype: object

## Se extraen los topicos de todos los restaurantes

In [17]:
stop_words = set(stopwords.words('english'))  # Puedes ajustar el idioma según tus datos
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if isinstance(text, str):  # Verificar si el texto es una cadena válida
        text = re.sub(r'\W', ' ', text)  # Eliminar caracteres no alfanuméricos
        tokens = word_tokenize(text.lower())  # Tokenización y minúsculas
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Lematización y eliminación de stopwords
        return tokens
    else:
        return []  # Devolver una lista vacía si no es una cadena válida

processed_reviews = [preprocess_text(review) for review in dfTacoBell["review_text"] if review is not None]

# Creación del diccionario y el corpus
dictionary = corpora.Dictionary(processed_reviews)
corpus = [dictionary.doc2bow(text) for text in processed_reviews]

# Modelado de tópicos usando LDA
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, random_state=42, passes=10)

# Visualización de tópicos
for idx, topic in lda_model.print_topics():
    print(f'Tópico {idx}: {topic}')

# Calcular la coherencia del modelo (opcional, para evaluar la calidad de los tópicos)
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_reviews, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print(f'\nCoherencia del modelo LDA: {coherence_lda}')

Tópico 0: 0.029*"food" + 0.016*"service" + 0.014*"order" + 0.011*"good" + 0.010*"fast" + 0.009*"got" + 0.008*"drive" + 0.008*"customer" + 0.007*"u" + 0.006*"back"
Tópico 1: 0.046*"taco" + 0.034*"bell" + 0.020*"food" + 0.013*"order" + 0.010*"get" + 0.009*"one" + 0.008*"time" + 0.007*"like" + 0.007*"drive" + 0.007*"good"
Tópico 2: 0.017*"drive" + 0.015*"order" + 0.013*"thru" + 0.013*"taco" + 0.011*"service" + 0.011*"bell" + 0.010*"food" + 0.009*"place" + 0.009*"time" + 0.009*"location"
Tópico 3: 0.024*"order" + 0.019*"time" + 0.014*"taco" + 0.013*"go" + 0.012*"food" + 0.011*"get" + 0.009*"bell" + 0.008*"drive" + 0.008*"employee" + 0.008*"always"
Tópico 4: 0.022*"taco" + 0.017*"order" + 0.016*"food" + 0.013*"time" + 0.012*"bell" + 0.009*"got" + 0.008*"one" + 0.008*"get" + 0.008*"service" + 0.008*"location"

Coherencia del modelo LDA: 0.44351581057761075


## Se realiza un modelo para predecir el promedio de estrellas de los restaurantes Taco Bell en los proximos años segun la ciudad

In [18]:


# Convertir la columna review_date a formato numérico
dfTacoBell['review_date'] = pd.to_datetime(dfTacoBell['review_date']).dt.strftime('%Y%m%d').astype(int)

# Filtrar los datos para los restaurantes Taco Bell
taco_bell_data = dfTacoBell[dfTacoBell['business_name'].str.contains('Taco Bell', case=False)]

# Seleccionar las características y la variable objetivo
features = ['review_date', 'city']
target = 'business_rating'

# Codificar la columna 'city' utilizando one-hot encoding
encoder = OneHotEncoder(handle_unknown='ignore')
city_encoded = encoder.fit_transform(taco_bell_data[['city']])

# Convertir el resultado de one-hot encoding a un DataFrame
city_encoded_df = pd.DataFrame(city_encoded.toarray(), columns=encoder.get_feature_names_out(['city']))

# Concatenar las características codificadas con las otras características
taco_bell_data_encoded = pd.concat([taco_bell_data[['review_date']].reset_index(drop=True), city_encoded_df], axis=1)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(taco_bell_data_encoded, taco_bell_data[target], test_size=0.2, random_state=42)

# Crear el modelo de regresión de bosque aleatorio
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

C:\Users\juanc.DESKTOP-LGMDQP1\AppData\Local\Temp\ipykernel_9780\287103178.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTacoBell['review_date'] = pd.to_datetime(dfTacoBell['review_date']).dt.strftime('%Y%m%d').astype(int)


Mean Squared Error: 0.18875514185720832


In [19]:
# Ejemplo de uso: predecir el business_rating para Taco Bell en los próximos años
new_data = pd.DataFrame({'review_date': [20230101, 20240101, 20250101], 'city': ['Miami', 'Orlando', 'Tampa']})

# Codificar la columna 'city' del nuevo conjunto de datos utilizando el mismo encoder
new_city_encoded = encoder.transform(new_data[['city']])

# Convertir el resultado de one-hot encoding a un DataFrame
new_city_encoded_df = pd.DataFrame(new_city_encoded.toarray(), columns=encoder.get_feature_names_out(['city']))

# Concatenar las características codificadas con la columna 'review_date'
new_data_encoded = pd.concat([new_data[['review_date']], new_city_encoded_df], axis=1)

# Asegurarse de que las columnas del nuevo conjunto de datos coincidan con las del conjunto de entrenamiento
missing_cols = set(X_train.columns) - set(new_data_encoded.columns)
for c in missing_cols:
    new_data_encoded[c] = 0
new_data_encoded = new_data_encoded[X_train.columns]

# Realizar predicciones con el modelo
new_predictions = model.predict(new_data_encoded)

print("Predicted business_rating for Taco Bell in the next years:")
for year, prediction in zip([2023, 2024, 2025], new_predictions):
    print(f"Year {year}: {prediction}")

Predicted business_rating for Taco Bell in the next years:
Year 2023: 2.21
Year 2024: 2.21
Year 2025: 1.71


## Se divide en dos dataframes las reseñas positivas y negativas de todos los restaurantes mexicanos de Florida

In [20]:
#Almacenamos en un df las reseñas positivas y en otro las negativas de todos los restaurantes mexicanos de Florida
filx=dfFinal["sentiment_analysis"] == 2
dfRestapositiv= dfFinal[filx] 

filz=dfFinal["sentiment_analysis"] == 0
dfRestanegat= dfFinal[filz] 

## Se divide en dos dataframes las reseñas positivas y negativas de todos los restaurantes Taco Bell de Florida

In [21]:
#Almacenamos en un df las reseñas positivas y en otro las negativas de todos los restaueantes de Taco Bell
filt=dfTacoBell["sentiment_analysis"] == 2
dfPostivTacoB= dfTacoBell[filt] 

film=dfTacoBell["sentiment_analysis"] == 0
dfNegativTacoB= dfTacoBell[film] 

## Se hace una extraccion de las palabaras mas repetidas en las reseñas positivas de todos los restaurantes mexicanos de Florida

In [22]:
#Hacemos una extraccion de las palabras mas relevantes en las reseñas positivas de los restaurantes mexicanos de florida

stop_words = set(stopwords.words('english'))  # Puedes ajustar el idioma según tus datos
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if isinstance(text, str):  # Verificar si el texto es una cadena válida
        text = re.sub(r'\W', ' ', text)  # Eliminar caracteres no alfanuméricos
        tokens = word_tokenize(text.lower())  # Tokenización y minúsculas
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Lematización y eliminación de stopwords
        return tokens
    else:
        return []  # Devolver una lista vacía si no es una cadena válida

processed_reviews = [preprocess_text(review) for review in dfRestapositiv["review_text"] if review is not None]

# Creación del diccionario y el corpus
dictionary = corpora.Dictionary(processed_reviews)
corpus = [dictionary.doc2bow(text) for text in processed_reviews]

# Modelado de tópicos usando LDA
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, random_state=42, passes=10)

# Visualización de tópicos
for idx, topic in lda_model.print_topics():
    print(f'Tópico {idx}: {topic}')

# Calcular la coherencia del modelo (opcional, para evaluar la calidad de los tópicos)
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_reviews, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print(f'\nCoherencia del modelo LDA: {coherence_lda}')

Tópico 0: 0.015*"great" + 0.013*"delicious" + 0.011*"fish" + 0.010*"shrimp" + 0.008*"also" + 0.008*"beach" + 0.008*"taco" + 0.008*"place" + 0.007*"beer" + 0.007*"drink"
Tópico 1: 0.020*"good" + 0.017*"chip" + 0.015*"salsa" + 0.014*"taco" + 0.012*"like" + 0.012*"chicken" + 0.010*"ordered" + 0.009*"burrito" + 0.008*"sauce" + 0.007*"rice"
Tópico 2: 0.017*"food" + 0.016*"order" + 0.015*"time" + 0.013*"u" + 0.010*"table" + 0.010*"get" + 0.008*"one" + 0.007*"go" + 0.007*"back" + 0.007*"minute"
Tópico 3: 0.080*"food" + 0.059*"great" + 0.041*"service" + 0.039*"good" + 0.025*"place" + 0.018*"friendly" + 0.016*"staff" + 0.015*"mexican" + 0.013*"always" + 0.012*"restaurant"
Tópico 4: 0.085*"taco" + 0.021*"best" + 0.021*"place" + 0.018*"mexican" + 0.014*"food" + 0.014*"original" + 0.013*"good" + 0.013*"delicious" + 0.013*"google" + 0.012*"translated"

Coherencia del modelo LDA: 0.40007278297505044


## Se hace una extraccion de las palabaras mas repetidas en las reseñas negativas de todos los restaurantes mexicanos de Florida

In [23]:
#Hacemos una extraccion de las palabras mas relevantes en las reseñas negativas de los restaurantes mexicanos de florida

stop_words = set(stopwords.words('english'))  # Puedes ajustar el idioma según tus datos
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if isinstance(text, str):  # Verificar si el texto es una cadena válida
        text = re.sub(r'\W', ' ', text)  # Eliminar caracteres no alfanuméricos
        tokens = word_tokenize(text.lower())  # Tokenización y minúsculas
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Lematización y eliminación de stopwords
        return tokens
    else:
        return []  # Devolver una lista vacía si no es una cadena válida

processed_reviews = [preprocess_text(review) for review in dfRestanegat["review_text"] if review is not None]

# Creación del diccionario y el corpus
dictionary = corpora.Dictionary(processed_reviews)
corpus = [dictionary.doc2bow(text) for text in processed_reviews]

# Modelado de tópicos usando LDA
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, random_state=42, passes=10)

# Visualización de tópicos
for idx, topic in lda_model.print_topics():
    print(f'Tópico {idx}: {topic}')

# Calcular la coherencia del modelo (opcional, para evaluar la calidad de los tópicos)
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_reviews, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print(f'\nCoherencia del modelo LDA: {coherence_lda}')


Tópico 0: 0.025*"taco" + 0.017*"food" + 0.014*"ordered" + 0.011*"chicken" + 0.010*"chip" + 0.010*"like" + 0.009*"good" + 0.009*"burrito" + 0.008*"salsa" + 0.008*"place"
Tópico 1: 0.050*"food" + 0.029*"place" + 0.024*"service" + 0.015*"good" + 0.011*"bad" + 0.010*"dirty" + 0.009*"go" + 0.009*"staff" + 0.008*"like" + 0.008*"mexican"
Tópico 2: 0.019*"la" + 0.016*"de" + 0.014*"original" + 0.014*"google" + 0.012*"translated" + 0.012*"el" + 0.009*"que" + 0.008*"e" + 0.008*"comida" + 0.007*"muy"
Tópico 3: 0.031*"order" + 0.017*"time" + 0.013*"food" + 0.012*"get" + 0.011*"taco" + 0.010*"location" + 0.009*"go" + 0.008*"customer" + 0.008*"one" + 0.008*"back"
Tópico 4: 0.019*"u" + 0.015*"food" + 0.013*"table" + 0.012*"minute" + 0.011*"drink" + 0.011*"service" + 0.010*"server" + 0.010*"came" + 0.010*"order" + 0.009*"back"

Coherencia del modelo LDA: 0.5452395354372408


## Se hace una extraccion de las palabaras mas repetidas en las reseñas positivas de todos los restaurantes Taco Bell de Florida

In [24]:
#Hacemos una extraccion de las palabras mas relevantes en las reseñas positivas de los restaurantes Taco Bell

stop_words = set(stopwords.words('english'))  # Puedes ajustar el idioma según tus datos
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if isinstance(text, str):  # Verificar si el texto es una cadena válida
        text = re.sub(r'\W', ' ', text)  # Eliminar caracteres no alfanuméricos
        tokens = word_tokenize(text.lower())  # Tokenización y minúsculas
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Lematización y eliminación de stopwords
        return tokens
    else:
        return []  # Devolver una lista vacía si no es una cadena válida

processed_reviews = [preprocess_text(review) for review in dfPostivTacoB["review_text"] if review is not None]

# Creación del diccionario y el corpus
dictionary = corpora.Dictionary(processed_reviews)
corpus = [dictionary.doc2bow(text) for text in processed_reviews]

# Modelado de tópicos usando LDA
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, random_state=42, passes=10)

# Visualización de tópicos
for idx, topic in lda_model.print_topics():
    print(f'Tópico {idx}: {topic}')

# Calcular la coherencia del modelo (opcional, para evaluar la calidad de los tópicos)
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_reviews, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print(f'\nCoherencia del modelo LDA: {coherence_lda}')

Tópico 0: 0.019*"taco" + 0.016*"food" + 0.014*"bell" + 0.013*"order" + 0.011*"drive" + 0.009*"time" + 0.008*"back" + 0.008*"one" + 0.007*"get" + 0.006*"like"
Tópico 1: 0.034*"taco" + 0.031*"food" + 0.023*"bell" + 0.020*"order" + 0.020*"good" + 0.015*"service" + 0.010*"always" + 0.009*"like" + 0.009*"get" + 0.009*"fast"
Tópico 2: 0.029*"taco" + 0.021*"bell" + 0.017*"food" + 0.016*"great" + 0.014*"order" + 0.012*"service" + 0.010*"staff" + 0.009*"time" + 0.009*"friendly" + 0.009*"always"
Tópico 3: 0.016*"taco" + 0.012*"bell" + 0.011*"food" + 0.011*"order" + 0.011*"one" + 0.009*"drive" + 0.008*"friendly" + 0.008*"always" + 0.007*"time" + 0.006*"get"
Tópico 4: 0.028*"taco" + 0.018*"bell" + 0.014*"food" + 0.011*"time" + 0.009*"order" + 0.009*"good" + 0.009*"one" + 0.008*"get" + 0.008*"chicken" + 0.007*"like"

Coherencia del modelo LDA: 0.3814056953052045


## Se hace una extraccion de las palabaras mas repetidas en las reseñas negativas de todos los restaurantes Taco Bell de Florida

In [25]:
#Hacemos una extraccion de las palabras mas relevantes en las reseñas negativas de los restaurantes Taco Bell

stop_words = set(stopwords.words('english'))  # Puedes ajustar el idioma según tus datos
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if isinstance(text, str):  # Verificar si el texto es una cadena válida
        text = re.sub(r'\W', ' ', text)  # Eliminar caracteres no alfanuméricos
        tokens = word_tokenize(text.lower())  # Tokenización y minúsculas
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Lematización y eliminación de stopwords
        return tokens
    else:
        return []  # Devolver una lista vacía si no es una cadena válida

processed_reviews = [preprocess_text(review) for review in dfNegativTacoB["review_text"] if review is not None]

# Creación del diccionario y el corpus
dictionary = corpora.Dictionary(processed_reviews)
corpus = [dictionary.doc2bow(text) for text in processed_reviews]

# Modelado de tópicos usando LDA
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, random_state=42, passes=10)

# Visualización de tópicos
for idx, topic in lda_model.print_topics():
    print(f'Tópico {idx}: {topic}')

# Calcular la coherencia del modelo (opcional, para evaluar la calidad de los tópicos)
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_reviews, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print(f'\nCoherencia del modelo LDA: {coherence_lda}')


Tópico 0: 0.016*"time" + 0.013*"food" + 0.012*"taco" + 0.011*"order" + 0.010*"bell" + 0.009*"service" + 0.008*"said" + 0.008*"customer" + 0.008*"get" + 0.007*"manager"
Tópico 1: 0.023*"taco" + 0.017*"order" + 0.017*"bell" + 0.015*"drive" + 0.013*"food" + 0.010*"thru" + 0.009*"minute" + 0.008*"get" + 0.008*"service" + 0.008*"time"
Tópico 2: 0.018*"food" + 0.011*"order" + 0.009*"got" + 0.009*"meat" + 0.008*"service" + 0.008*"never" + 0.007*"asked" + 0.007*"even" + 0.007*"taco" + 0.007*"said"
Tópico 3: 0.025*"taco" + 0.022*"bell" + 0.020*"food" + 0.017*"order" + 0.015*"get" + 0.014*"go" + 0.014*"time" + 0.013*"place" + 0.010*"never" + 0.009*"location"
Tópico 4: 0.025*"taco" + 0.020*"order" + 0.014*"food" + 0.013*"bell" + 0.011*"time" + 0.010*"one" + 0.010*"back" + 0.010*"got" + 0.009*"get" + 0.007*"location"

Coherencia del modelo LDA: 0.3678846864288593


In [1]:
import streamlit as st

In [29]:
st.write("Hello World")

2024-07-02 12:09:33.953 
  command:

    streamlit run C:\Users\juanc.DESKTOP-LGMDQP1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
